In [1]:
# Bringing the data from kaggle and unzipping
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/AIDocAssist/fracture_detection/kaggle.json ~/.kaggle/
!kaggle datasets download -d srinivasvishal7/fracture
!unzip /content/fracture.zip

 48% 9.00M/18.8M [00:00<00:00, 42.3MB/s]
100% 18.8M/18.8M [00:00<00:00, 61.3MB/s]
Archive:  /content/fracture.zip
  inflating: fracture/training/0.jpg  
  inflating: fracture/training/1.jpg  
  inflating: fracture/training/10.jpg  
  inflating: fracture/training/100.jpg  
  inflating: fracture/training/101.jpg  
  inflating: fracture/training/102.jpg  
  inflating: fracture/training/103.jpg  
  inflating: fracture/training/104.jpg  
  inflating: fracture/training/105.jpg  
  inflating: fracture/training/106.jpg  
  inflating: fracture/training/107.jpg  
  inflating: fracture/training/108.jpg  
  inflating: fracture/training/109.jpg  
  inflating: fracture/training/11.jpg  
  inflating: fracture/training/110.jpg  
  inflating: fracture/training/111.jpg  
  inflating: fracture/training/112.jpg  
  inflating: fracture/training/113.jpg  
  inflating: fracture/training/114.jpg  
  inflating: fracture/training/115.jpg  
  inflating: fracture/training/116.jpg  
  inflating: fracture/training/

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import os, glob
import cv2
import tensorflow as tf

In [3]:
def xml_to_csv(path, pathImg):
    xml_list = []
    i=0
    for xml_file in glob.glob(path + '/*.xml'):
        
        tree = ET.parse(xml_file)
        root = tree.getroot()
        img = tf.io.decode_jpeg(tf.io.read_file(pathImg+"/"+root.find('filename').text))
        im_width, im_height = img.shape[1], img.shape[0]
        for member in root.findall('object'):
            center_x = (int(member[4][0].text)+int(member[4][2].text))/2
            center_y = (int(member[4][1].text)+int(member[4][3].text))/2
            b_width = (int(member[4][2].text) - int(member[4][0].text))
            b_height = (int(member[4][3].text) - int(member[4][1].text))
            value = (pathImg.split("/")[-1]+"/"+root.find('filename').text,
                     im_width,
                     im_height,
                     center_x,
                     center_y,
                     b_width,
                     b_height
                     )
            
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'center_x', 'center_y', 'b_width', 'b_height']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [4]:
train_df = xml_to_csv("/content/fracture/training_annotations", "/content/fracture/training")
validation_df = xml_to_csv("/content/fracture/validation_annotations", "/content/fracture/validation")

In [7]:
train_df[:60]

,filename,width,height,center_x,center_y,b_width,b_height
0,training/229.jpg,800,1093,0.469375,0.689387,0.098750,0.070448
1,training/140.jpg,800,1093,0.458750,0.682983,0.117500,0.064959
2,training/129.jpg,555,865,0.273874,0.471676,0.064865,0.030058
3,training/92.jpg,702,1200,0.772080,0.642500,0.116809,0.070000
4,training/230.jpg,1024,1024,0.660156,0.675781,0.080078,0.097656
5,training/161.jpg,555,865,0.277477,0.476301,0.054054,0.039306
6,training/138.jpg,245,450,0.406122,0.414444,0.134694,0.122222
7,training/106.jpg,527,752,0.715370,0.699468,0.106262,0.090426
8,training/209.jpg,345,768,0.643478,0.655599,0.139130,0.066406
9,training/94.jpg,298,667,0.194631,0.771364,0.288591,0.127436


In [6]:
train_df['center_x'] = train_df["center_x"]/train_df['width']
train_df['b_width'] = train_df["b_width"]/train_df['width']
train_df['center_y'] = train_df["center_y"]/train_df['height']
train_df['b_height'] = train_df["b_height"]/train_df['height']
validation_df['center_x'] = validation_df["center_x"]/validation_df['width']
validation_df['b_width'] = validation_df["b_width"]/validation_df['width']
validation_df['center_y'] = validation_df["center_y"]/validation_df['height']
validation_df['b_height'] = validation_df["b_height"]/validation_df['height']

In [8]:
train_df.to_csv("/content/drive/MyDrive/AIDocAssist/fracture_detection/train_df2.csv",index=False)
validation_df.to_csv("/content/drive/MyDrive/AIDocAssist/fracture_detection/validation_df2.csv",index=False)